# GHOST (General Holistic Organism Scripting Tool)

# Goals and Vision

The design intention of Ghost (the “General Holistic Organism Scripting Tool”) is to allow human authors to script behaviors for artificial characters.   Significant flexibility is desired, e.g. to support
* Purely textual chatbots, animated characters, or physical robots
* Characters controlled precisely by human-authored rules; or characters with a great deal of autonomy, where human-authored rules serve mainly to tweak parameters and indicate propensities

Ghost is envisioned as having both a textual and a graphical user interface.   The textual interface is aimed to have roughly the same level of complexity as ChatScript (on which it is heavily based), meaning that it should be usable by non-programmers who are able to deal with a fairly intricate formal syntax. 

The GUI is not yet designed at time of writing, but is intended to make Ghost authoring feasible for a broader class of content authors, and to make it easier and more rapid for everyone.   Some brief notes  regarding the envisioned GUI are given at the end of this document.

# High Level Design

Ghost is implemented as a DSL (Domain-Specific Language) within the Scheme shell associated with the OpenCog engine.    Ghost syntax closely resembles (and in most respects is identical to) ChatScript; but the Ghost interpreter is written in Scheme and runs in the OpenCog Scheme shell.   Ghost rules are interpreted into Atoms living in OpenCog’s Atomspace semantic knowledge store, and are executed within OpenCog using the OpenCog Pattern Matcher and Action Selector and associated mechanisms.

A Ghost rule has two required parts, a pattern and an action; and an optional third part, a goal or set of goals.   The general semantic is: When the pattern is observed, if the goal is being pursued by the agent, then consider taking the action. If no goal is specified, a generic goal such as “interact with others” is implicitly assumed. A Ghost “script” consists of a set of rules, which may be grouped into files called “topics.”   A topic may have some annotations associated with it, which apply to all the rules in the topic (unless overridden by annotations specific to certain rules.)

Both patterns and actions may optionally reference Scheme functions.  (Support for functions written in other languages may be added in a later version.)   An envisioned usage pattern is: Writing Ghost rules and, in coordination with the Ghost rule authoring process, adding new Scheme functions to help the rules do their things.

An example simple Ghost rule is:
```
#goal: (novelty=1)
s: ( Hi there ^no_people_around ) Who said that? ^look_around
```
This rule is somewhat trivial as it lacks variables.   But it shows the basic format; here we have:
 * Pattern = someone says “Hi there” to the agent, and there are no people around (i.e. the Scheme function no_people_around outputs True)
 * Action = the agent says “Who said that?” and then invokes the Scheme function look_around
 * Goal = novelty … so the agent should use this rule when the “novelty” goal is important to it
 
Behind the scenes, the Ghost interpreter uses OpenCog’s Pattern Matcher to match the patterns of its rules to content in OpenCog’s Atomspace knowledge store.  The specifics of this matching are simple now, but are highly customizable and can be made more sophisticated in future versions.

